In [3]:
import json
import pandas as pd
from google.cloud import bigquery
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd

import re
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle


In [4]:
client = bigquery.Client(project="ingka-sot-cfm-dev")


/Users/salvatore.lovecchio/Library/Python/3.9/lib/python/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
#Code related with the source to be extracted
code=""" 

  with item_description as (
    SELECT
  it.item_no,
  it.item_name,
  it.pa_no,
  it.pa_name,
  it.hfb_name,
  it.hfb_no,
FROM
  `ingka-rrm-erix-prod.erix_legacy_production.item_dim` it
where hfb_name <> 'UNKNOWN'
  ), 
  grid_location_item as (
  SELECT
    article_id,
    bu_code,
    sales_location_id,
    CASE
      WHEN location_type = '1' THEN 'H'
      WHEN location_type = '2' THEN 'P'
      WHEN location_type = '3' THEN 'HP'
      WHEN location_type = '0' THEN 'Addons'
      ELSE 'Other'
  END
    AS location_type_derived,
    location_type,
    assigned_sales_space_quantity,
    PARSE_DATE('%Y%m%d', from_date) AS from_date,
    # nex import_date is the until_date with the condition of row_numb = 1
    import_date as till_date,
    active,
ROW_NUMBER() OVER (PARTITION BY article_id, bu_code, sales_location_id 
ORDER BY CAST(import_date AS DATE) DESC) AS row_num
  FROM
    `ingka-cff-slm-prod.slm.article_location_weekly`
  WHERE
 bu_code ="445"
   --and  import_date < '2025-09-14'
      and  import_date >= '2025-01-24'
and active ='1'
 Qualify row_num = 1
  ),
  grid_item_dep_match as(
select *,
  REGEXP_EXTRACT(UPPER(TRIM(sales_location_id)), r'^(?:S|M)(\d{2})') AS dept_2d,
    case when REGEXP_EXTRACT(UPPER(TRIM(sales_location_id)), r'^(?:S|M)(\d{2})') = hfb_no then 'match'
       else 'no match'
    end as dept_matches_hfb
 from grid_location_item as grid
  left join
  item_description as item
    on grid.article_id = item.item_no
  )
  select *
  from grid_item_dep_match

"""

job=client.query(code)

location_exploration=job.to_dataframe()

E0000 00:00:1762344475.771910 20285845 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [11]:
location_exploration.to_csv('location_exploration.csv', index=False, encoding='utf-8')


In [12]:
location_exploration = pd.read_csv("location_exploration.csv")

location_exploration['till_date'] = pd.to_datetime(location_exploration['till_date'])
location_exploration['from_date'] = pd.to_datetime(location_exploration['from_date'])


In [16]:
### filter 

filter_current = location_exploration.loc[location_exploration.till_date=='2025-11-05']
filter_current.head(20)

,article_id,bu_code,sales_location_id,location_type_derived,location_type,assigned_sales_space_quantity,from_date,till_date,active,row_num,item_no,item_name,pa_no,pa_name,hfb_name,hfb_no,dept_2d,dept_matches_hfb
0,20582182,445,M06E20,Addons,0,3.0,2025-10-20,2025-11-05,1,1,20582182,BROFJÄRDEN waste bin 4 l chrome effect,623,"Bathroom, shower and toilet solutions",Bathroom & Water,6,6.0,match
2,40421051,445,051710,HP,3,20.0,2024-10-23,2025-11-05,1,1,40421051,ME frm wll cb 30x37x60 white,711,"Kitchen cabinets, kitchen legs, open frames",Kitchen,7,NaN,no match
7,40454008,445,M06D24,HP,3,102.0,2025-05-25,2025-11-05,1,1,40454008,KROKFJORDEN shwr hngr 2 tiers 24x53 zinc plated,623,"Bathroom, shower and toilet solutions",Bathroom & Water,6,6.0,match
8,60163862,445,M10K12,HP,3,13.0,2025-05-02,2025-11-05,1,1,60163862,ÅRSTID N floor lamp nickel-plated/white,1011,Floor lamps,Lighting & Home electronics,10,10.0,match
10,90368800,445,571410,Addons,0,3.0,2021-10-20,2025-11-05,1,1,90368800,MATÄLSKARE wll mount extr hood 60 stainless steel,7015,Extractor hoods,Home Appliances,70,NaN,no match
11,90582027,445,020900,HP,3,27.0,2025-10-12,2025-11-05,1,1,90582027,EKET media shelf 70x35x35 white,231,TV & Media Furniture,Store and organise furniture,2,NaN,no match
14,50474770,445,M18G17,Addons,0,8.0,2025-06-02,2025-11-05,1,1,50474770,ELLOVEN monitor stand with drawer white,361,Complementary workspaces items,Workspaces,3,18.0,no match
15,90205605,445,053810,HP,3,50.0,2022-01-18,2025-11-05,1,1,90205605,UTRUSTA shelf 60x37 glass 2-p,731,Kitchen fixed interiors,Kitchen,7,NaN,no match
17,70205531,445,520200,Addons,0,36.0,2024-05-14,2025-11-05,1,1,70205531,ME frm wll cb 40x37x80 white,711,"Kitchen cabinets, kitchen legs, open frames",Kitchen,7,NaN,no match
19,494025,445,M06E20,HP,3,30.0,2025-10-13,2025-11-05,1,1,494025,TOFTAN N waste bin 4 l white,623,"Bathroom, shower and toilet solutions",Bathroom & Water,6,6.0,match


In [20]:
filter_current.loc[filter_current.hfb_no==1]

,article_id,bu_code,sales_location_id,location_type_derived,location_type,assigned_sales_space_quantity,from_date,till_date,active,row_num,item_no,item_name,pa_no,pa_name,hfb_name,hfb_no,dept_2d,dept_matches_hfb
68,60540227,445,550815,HP,3,5.0,2025-01-27,2025-11-05,1,1,60540227,KIVIK cvr f sb1 Kelinge grey-turquoise,113,Sofa beds,Living room seating,1,NaN,no match
268,60466699,445,PALLET,HP,3,5.0,2024-03-15,2025-11-05,1,1,60466699,VIMLE frm sb2 sec Grann/Bomstad black,113,Sofa beds,Living room seating,1,NaN,no match
364,20546622,445,531415,HP,3,2.0,2025-10-02,2025-11-05,1,1,20546622,NYHAMN cvr f sb3 Skartofta black/light grey,113,Sofa beds,Living room seating,1,NaN,no match
542,60489549,445,680400,HP,3,6.0,2022-03-19,2025-11-05,1,1,60489549,VIMLE sb2,113,Sofa beds,Living room seating,1,NaN,no match
674,90545186,445,660610,HP,3,6.0,2023-11-07,2025-11-05,1,1,90545186,VRETSTORP cvr f sb3 Kilanda light beige,113,Sofa beds,Living room seating,1,NaN,no match
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9285,50494626,445,170810,Addons,0,4.0,2023-10-11,2025-11-05,1,1,50494626,STRANDMON fst Grann/Bomstad dark brown,126,Footstools,Living room seating,1,NaN,no match
9288,90525961,445,433710,HP,3,10.0,2025-09-02,2025-11-05,1,1,90525961,EMMISLÖV pouffe Vissle dark grey,126,Footstools,Living room seating,1,NaN,no match
9290,50494626,445,170710,HP,3,6.0,2025-10-02,2025-11-05,1,1,50494626,STRANDMON fst Grann/Bomstad dark brown,126,Footstools,Living room seating,1,NaN,no match
9292,60492858,445,190810,HP,3,18.0,2025-09-05,2025-11-05,1,1,60492858,POÄNG csh ftst Skiftebo dark grey,126,Footstools,Living room seating,1,NaN,no match


In [17]:
import pandas as pd
import numpy as np
import re

df = filter_current.copy()

# --- setup ---
sales_loc_col = "sales_location_id"  # or "c" if that's your column
loc_upper  = df["location_type_derived"].astype(str).str.strip().str.upper()
dept_match = df["dept_matches_hfb"].astype(str).str.strip()

# Flags (match your SQL: ^H OR contains P anywhere)
df["is_h_hp_p"]    = loc_upper.str.startswith("H") | loc_upper.str.contains("P")
df["is_addons"]    = loc_upper.eq("ADDONS")
df["is_same_dep"]  = dept_match.eq("match")
df["is_other_dep"] = dept_match.eq("no match")
df["is_dept_null"] = df["dept_2d"].isna()

# --- group & aggregate (distinct counts) ---
grouped_item = df.groupby("hfb_no").apply(lambda g: pd.Series({
    "numb_art":                         g["article_id"].nunique(),
    "sales_location":                   g[sales_loc_col].nunique(),
    "n_articles_h_or_hp_or_p":          g.loc[g["is_h_hp_p"],    "article_id"].nunique(),
    "n_articles_addons":                g.loc[g["is_addons"],    "article_id"].nunique(),
    "n_articles_Other_dep":             g.loc[g["is_other_dep"], "article_id"].nunique(),
    "n_articles_same_dep":              g.loc[g["is_same_dep"],  "article_id"].nunique(),
    "distinct_dept":                    g["dept_2d"].nunique(dropna=True),
    "n_articles_not_Show_OR_Mh":        g.loc[g["is_dept_null"], "article_id"].nunique(),
})).reset_index()

/tmp/ipykernel_63980/919950941.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_item = df.groupby("hfb_no").apply(lambda g: pd.Series({


In [18]:
grouped_item

,hfb_no,numb_art,sales_location,n_articles_h_or_hp_or_p,n_articles_addons,n_articles_Other_dep,n_articles_same_dep,distinct_dept,n_articles_not_Show_OR_Mh
0,1,770,614,770,71,770,2,1,770
1,2,671,615,671,73,671,2,2,670
2,3,346,369,346,71,338,24,3,334
3,4,762,805,762,156,761,15,2,761
4,5,395,494,395,150,395,0,2,387
5,6,454,328,454,64,338,144,8,317
6,7,663,525,663,93,661,38,6,647
7,8,335,383,335,63,335,1,2,335
8,9,613,370,613,171,374,351,15,315
9,10,650,158,650,184,199,560,12,120


In [19]:
grouped_item.to_csv('grouped_item.csv', index=False, encoding='utf-8')
